In [58]:
import torch
import os
import numpy as np
import pandas as pd
from warnings import simplefilter
from torch.utils.data import DataLoader

simplefilter(action='ignore', category=UserWarning)
simplefilter(action='ignore', category=FutureWarning)

In [2]:
MAIN_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
DATA_DIR = os.path.join(MAIN_DIR, "Solid_droplet", "Data")

In [3]:
from Dataloader import *
from Trainer import *
from model import *

In [4]:
dtype = torch.float32
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
# Split the data into train, validation and test set
dataspliter = DataSpliter(DATA_DIR, device, train_frac=0.6, val_frac=0.2, test_frac=0.2, use_seed=True, seed_val=42)

# Load Data from DataLoader
train_set = dataspliter.train
val_set = dataspliter.val
test_set = dataspliter.test

train_dataset = CustomImageDataset(DATA_DIR, train_set)
val_dataset = CustomImageDataset(DATA_DIR, val_set)
test_dataset = CustomImageDataset(DATA_DIR, test_set)

In [6]:
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=True)

In [7]:
# Get the model
model = CNNModel()
trainer = Trainer(model, train_dataloader, val_dataloader, test_dataloader)

The device that will be used in training is Quadro P1000


In [8]:
trainer.fit(epochs=10, batch_size= 8)
trainer.save_model("model.pt", "savefolderpytorch")

Epoch 1


Training: 27it [00:04,  5.47it/s, loss=560]                 
Validation: 9it [00:00,  9.67it/s, loss=559]               

Validation loss is 194.25652694702148
Epoch 2



Training: 27it [00:03,  8.21it/s, loss=610]             
Validation: 9it [00:00,  9.24it/s, loss=232]                   

Validation loss is 313.79787826538086
Epoch 3


...

Validation loss is 80.85004425048828
Epoch 9



Training: 27it [00:03,  8.21it/s, loss=44.5]              
Validation: 9it [00:01,  8.36it/s, loss=30]                

Validation loss is 61.15915393829346
Epoch 10



Training: 27it [00:03,  8.74it/s, loss=12.8]              
Validation: 9it [00:01,  8.44it/s, loss=337]               

Validation loss is 120.12248992919922


In [67]:
_, output, labels = trainer.val_epoch(test_dataloader)

Validation: 9it [00:01,  7.98it/s, loss=88.3]               

Validation loss is 32.01697063446045


In [62]:
def list_of_arr_to_arr(list_of_arr):
    for i in range(1, len(output)):
        if i == 1:
            output_arr = np.append(output[0],output[i])
        else:
            output_arr = np.append(output_arr, output[i])
    output_arr.reshape(int(output_arr.shape[0]/3),3)
    return output_arr

TypeError: append() missing 1 required positional argument: 'values'

ValueError: cannot reshape array of size 27 into shape (3,3)

array([[35.53734  , 33.70323  ,  2.7987587],
       [39.695263 , 36.654163 ,  2.5160363],
       [31.990023 , 30.383936 ,  3.6535256],
       [39.856586 , 37.5989   ,  4.4827995],
       [41.17421  , 31.033033 ,  3.1219618],
       [47.73812  , 35.32497  ,  4.144422 ],
       [27.950716 , 28.21852  ,  1.9211624],
       [41.747116 , 40.898575 ,  1.8264115],
       [33.165123 , 34.087772 ,  4.1424675]], dtype=float32)

9